In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('telecom_clean.csv')

In [14]:
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
2,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No
4,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5981,Male,0,Yes,No,1,Yes,No,Fiber optic,Yes,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.00,95.00,Yes
5982,Female,0,Yes,Yes,23,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),91.10,2198.30,No
5983,Male,0,Yes,Yes,12,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,21.15,306.05,No
5984,Male,1,No,No,12,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.45,1200.15,Yes


In [15]:
X = df.drop(columns=['Churn'])
y = df['Churn']

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Pipeline

In [17]:
from sklearn.base import TransformerMixin, BaseEstimator

class DTypeSelector(BaseEstimator, TransformerMixin):

    def __init__(self, dtypes):
        self.dtypes = dtypes


    def fit(self, X, y):
        return self


    def transform(self, X):
        return X.select_dtypes(self.dtypes)

In [18]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [19]:
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [23]:
import keras

In [30]:
def create_model(optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy', keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall')])
    
    return model

In [31]:
# def create_model(optimizer='adam'):
#     model = Sequential()
    
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
    
#     model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy', keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall')])
    
#     return model

In [46]:
pipeline = make_pipeline(
    make_union(
        make_pipeline(
        DTypeSelector('number'),
        SimpleImputer(),
        StandardScaler()
    ),
        make_pipeline(
        DTypeSelector('object'),
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder()
    )
    ),
    KerasClassifier(build_fn=create_model,verbose=1, epochs=100, batch_size=32)
)

In [45]:
pipeline.fit(X_train, y_train)

ValueError: Expected `class_weight` to be a dict with keys from 0 to one less than the number of classes, found {'Yes': 0.9, 'No': 0.1}

In [34]:
y_preds = pipeline.predict(X_test)
y_preds_training = pipeline.predict(X_train)

126/126 [==============================] - 0s 623us/step


In [35]:
from sklearn.metrics import classification_report, confusion_matrix

In [36]:
print(classification_report(y_train, y_preds_training))

              precision    recall  f1-score   support

          No       0.86      0.88      0.87      2931
         Yes       0.64      0.60      0.62      1079

    accuracy                           0.80      4010
   macro avg       0.75      0.74      0.74      4010
weighted avg       0.80      0.80      0.80      4010



In [37]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

          No       0.86      0.89      0.87      1468
         Yes       0.63      0.57      0.60       508

    accuracy                           0.80      1976
   macro avg       0.74      0.73      0.74      1976
weighted avg       0.80      0.80      0.80      1976



# Weight balancing

In [63]:
df['Churn'].unique()

array(['No', 'Yes'], dtype=object)

In [64]:
class_weights = {0: 0.1, 1: 0.9}

In [65]:
def create_model(optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy', keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall')])
    
    return model

In [66]:
pipeline = make_pipeline(
    make_union(
        make_pipeline(
        DTypeSelector('number'),
        SimpleImputer(),
        StandardScaler()
    ),
        make_pipeline(
        DTypeSelector('object'),
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder()
    )
    ),
    KerasClassifier(build_fn=create_model,verbose=1, epochs=100, batch_size=32, class_weight=class_weights)
)

In [67]:
pipeline.fit(X_train, y_train)

Epoch 1/100
126/126 [==============================] - 1s 1ms/step - loss: 3.1158 - accuracy: 0.2915 - precision: 0.2774 - recall: 0.9710
Epoch 2/100
126/126 [==============================] - 0s 1ms/step - loss: 0.4048 - accuracy: 0.2665 - precision: 0.2665 - recall: 1.0000
Epoch 3/100
126/126 [==============================] - 0s 2ms/step - loss: 0.1839 - accuracy: 0.2927 - precision: 0.2746 - recall: 0.9974
Epoch 4/100
126/126 [==============================] - 0s 2ms/step - loss: 0.1740 - accuracy: 0.4461 - precision: 0.3293 - recall: 0.9804
Epoch 5/100
126/126 [==============================] - 0s 2ms/step - loss: 0.1678 - accuracy: 0.5144 - precision: 0.3514 - recall: 0.9741
Epoch 6/100
126/126 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.4699 - precision: 0.3427 - recall: 0.9832
Epoch 7/100
126/126 [==============================] - 0s 2ms/step - loss: 0.1657 - accuracy: 0.4665 - precision: 0.3189 - recall: 0.9773
Epoch 8/100
126/126 [=============

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('pipeline-1',
                                                 Pipeline(memory=None,
                                                          steps=[('dtypeselector',
                                                                  DTypeSelector(dtypes='number')),
                                                                 ('simpleimputer',
                                                                  SimpleImputer(add_indicator=False,
                                                                                copy=True,
                                                                                fill_value=None,
                                                                                missing_values=nan,
                                                                                strategy='mean',
                 

In [68]:
y_preds = pipeline.predict(X_test)
y_preds_training = pipeline.predict(X_train)

126/126 [==============================] - 0s 638us/step


In [69]:
print(classification_report(y_train, y_preds_training))

              precision    recall  f1-score   support

          No       0.94      0.60      0.73      2931
         Yes       0.45      0.90      0.60      1079

    accuracy                           0.68      4010
   macro avg       0.69      0.75      0.66      4010
weighted avg       0.81      0.68      0.70      4010



In [70]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

          No       0.93      0.59      0.72      1468
         Yes       0.43      0.88      0.57       508

    accuracy                           0.67      1976
   macro avg       0.68      0.74      0.65      1976
weighted avg       0.80      0.67      0.69      1976



# Focal loss

In [72]:
import keras
from keras import backend as K
import tensorflow as tf

# Define our custom loss function
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [75]:
def create_model(optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=[focal_loss],optimizer=optimizer, metrics=['accuracy', keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall')])
    
    return model

In [76]:
pipeline = make_pipeline(
    make_union(
        make_pipeline(
        DTypeSelector('number'),
        SimpleImputer(),
        StandardScaler()
    ),
        make_pipeline(
        DTypeSelector('object'),
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder()
    )
    ),
    KerasClassifier(build_fn=create_model,verbose=1, epochs=100, batch_size=32)
)

In [77]:
pipeline.fit(X_train, y_train)

Epoch 1/100
126/126 [==============================] - 1s 1ms/step - loss: 5.7616 - accuracy: 0.7303 - precision: 0.2402 - recall: 5.8889e-04
Epoch 2/100
126/126 [==============================] - 0s 1ms/step - loss: 3.5933 - accuracy: 0.7427 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/100
126/126 [==============================] - 0s 1ms/step - loss: 2.7425 - accuracy: 0.7251 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/100
126/126 [==============================] - 0s 1ms/step - loss: 2.2419 - accuracy: 0.7279 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/100
126/126 [==============================] - 0s 1ms/step - loss: 1.9773 - accuracy: 0.7392 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/100
126/126 [==============================] - 0s 1ms/step - loss: 1.8115 - accuracy: 0.7335 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/100
126/126 [==============================] - 0s 1ms/step - loss: 1.7173 - accuracy: 0.7262 - precision: 0.0000e+00 - r

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('pipeline-1',
                                                 Pipeline(memory=None,
                                                          steps=[('dtypeselector',
                                                                  DTypeSelector(dtypes='number')),
                                                                 ('simpleimputer',
                                                                  SimpleImputer(add_indicator=False,
                                                                                copy=True,
                                                                                fill_value=None,
                                                                                missing_values=nan,
                                                                                strategy='mean',
                 

In [78]:
y_preds = pipeline.predict(X_test)
y_preds_training = pipeline.predict(X_train)

126/126 [==============================] - 0s 614us/step


In [79]:
print(classification_report(y_train, y_preds_training))

              precision    recall  f1-score   support

          No       0.73      1.00      0.84      2931
         Yes       0.00      0.00      0.00      1079

    accuracy                           0.73      4010
   macro avg       0.37      0.50      0.42      4010
weighted avg       0.53      0.73      0.62      4010



In [80]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

          No       0.74      1.00      0.85      1468
         Yes       0.00      0.00      0.00       508

    accuracy                           0.74      1976
   macro avg       0.37      0.50      0.43      1976
weighted avg       0.55      0.74      0.63      1976



# Custom loss function

In [116]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

In [117]:
def create_model(optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=metrics)
    
    return model

In [130]:
class_weight = {0: 0.3, 1: 0.7}

In [131]:
pipeline = make_pipeline(
    make_union(
        make_pipeline(
        DTypeSelector('number'),
        SimpleImputer(),
        StandardScaler()
    ),
        make_pipeline(
        DTypeSelector('object'),
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder()
    )
    ),
    KerasClassifier(build_fn=create_model,verbose=1, epochs=100, batch_size=32, class_weight=class_weight)
)

In [132]:
pipeline.fit(X_train, y_train)

Epoch 1/100
126/126 [==============================] - 2s 2ms/step - loss: 3.4597 - fn: 306.2756 - fp: 1399.7087 - tn: 3031.5433 - tp: 1319.8740 - precision: 0.4845 - recall: 0.8151
Epoch 2/100
126/126 [==============================] - 0s 2ms/step - loss: 0.3208 - fn: 136.0551 - fp: 370.7717 - tn: 1118.8268 - tp: 421.7480 - precision: 0.5334 - recall: 0.7625
Epoch 3/100
126/126 [==============================] - 0s 2ms/step - loss: 0.2433 - fn: 124.4567 - fp: 373.6063 - tn: 1110.0315 - tp: 439.3071 - precision: 0.5371 - recall: 0.7897
Epoch 4/100
126/126 [==============================] - 0s 2ms/step - loss: 0.2335 - fn: 132.4961 - fp: 347.6142 - tn: 1147.7953 - tp: 419.4961 - precision: 0.5501 - recall: 0.7545
Epoch 5/100
126/126 [==============================] - 0s 2ms/step - loss: 0.2253 - fn: 125.1969 - fp: 356.5906 - tn: 1159.9528 - tp: 405.6614 - precision: 0.5305 - recall: 0.7466
Epoch 6/100
126/126 [==============================] - 0s 2ms/step - loss: 0.2331 - fn: 132.6220 -

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('pipeline-1',
                                                 Pipeline(memory=None,
                                                          steps=[('dtypeselector',
                                                                  DTypeSelector(dtypes='number')),
                                                                 ('simpleimputer',
                                                                  SimpleImputer(add_indicator=False,
                                                                                copy=True,
                                                                                fill_value=None,
                                                                                missing_values=nan,
                                                                                strategy='mean',
                 

In [133]:
y_preds = pipeline.predict(X_test)
y_preds_training = pipeline.predict(X_train)

126/126 [==============================] - 0s 743us/step


In [134]:
print(classification_report(y_train, y_preds_training))

              precision    recall  f1-score   support

          No       0.88      0.81      0.84      2931
         Yes       0.58      0.71      0.64      1079

    accuracy                           0.78      4010
   macro avg       0.73      0.76      0.74      4010
weighted avg       0.80      0.78      0.79      4010



In [135]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

          No       0.88      0.82      0.85      1468
         Yes       0.56      0.67      0.61       508

    accuracy                           0.78      1976
   macro avg       0.72      0.74      0.73      1976
weighted avg       0.79      0.78      0.78      1976



# Optuna 

In [250]:
import keras.backend as K

import optuna

from optkeras.optkeras import OptKeras

In [251]:
ok = OptKeras(study_name='test')

[I 2021-05-07 15:01:59,702] A new study created in memory with name: test


[2021-05-07 15:01:59.705541] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


In [253]:
def create_model(units1, activation, dropout, optimizer='adam'):
    model = Sequential()
    
    model.add(Dense(256, activation=activation, kernel_regularizer='l2'))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation=activation, kernel_regularizer='l2'))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation=activation, kernel_regularizer='l2'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=metrics)
    
    return model

In [254]:
def create_pipeline(units1, activation, dropout):
    return make_pipeline(
        make_union(
            make_pipeline(
                DTypeSelector('number'),
                SimpleImputer(),
                StandardScaler()
            ),
            make_pipeline(
                DTypeSelector('object'),
                SimpleImputer(strategy='most_frequent'),
                OneHotEncoder()
            )
        ),
        KerasClassifier(build_fn=create_model, units1=units1, activation=activation, dropout=dropout,verbose=1, epochs=100, batch_size=32, class_weight=class_weight)

    )

In [255]:
pipeline1 = create_pipeline(100, 'relu', 0.3)

In [261]:
tf.convert_to_tensor(y_train)

<tf.Tensor: shape=(4010,), dtype=string, numpy=array([b'No', b'No', b'No', ..., b'No', b'No', b'No'], dtype=object)>

In [266]:
tf.convert_to_tensor(X_train[['gender','Partner']])

<tf.Tensor: shape=(4010, 2), dtype=string, numpy=
array([[b'Male', b'Yes'],
       [b'Male', b'Yes'],
       [b'Male', b'No'],
       ...,
       [b'Male', b'No'],
       [b'Female', b'Yes'],
       [b'Female', b'Yes']], dtype=object)>

In [270]:
tf.convert_to_tensor(X_train[['MonthlyCharges']])

<tf.Tensor: shape=(4010, 1), dtype=float64, numpy=
array([[90.8 ],
       [69.1 ],
       [50.75],
       ...,
       [63.95],
       [75.2 ],
       [99.  ]])>

In [271]:
import optuna

def objective(trial):
    
    K.clear_session()
    
    pipeline = create_model(
        units1=trial.suggest_uniform('units1', 50, 100),
        activation=trial.suggest_categorical('activation', ['relu', 'linear']),
        dropout=trial.suggest_uniform('dropout', 0.1, 0.4)
    )
    
    pipeline.fit(X_train, y_train)
    
    return ok.trial_best_value

In [272]:
ok.optimize(objective, timeout=60)

[W 2021-05-07 15:33:30,745] Trial 1 failed because of the following error: ValueError('Failed to convert a NumPy array to a Tensor (Unsupported object type int).')
Traceback (most recent call last):
  File "/Users/nielshoogeveen/Desktop/Projects/auto_autoML/venv/lib/python3.7/site-packages/tensorflow/python/data/util/structure.py", line 106, in normalize_element
    spec = type_spec_from_value(t, use_fallback=False)
  File "/Users/nielshoogeveen/Desktop/Projects/auto_autoML/venv/lib/python3.7/site-packages/tensorflow/python/data/util/structure.py", line 480, in type_spec_from_value
    (element, type(element).__name__))
TypeError: Could not build a TypeSpec for       gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
4336    Male              0     Yes        Yes      72          Yes   
3253    Male              0     Yes         No      59          Yes   
5241    Male              0      No         No       1          Yes   
3181    Male              0     Yes        Yes

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).